# This notebook is used to extract fixations for each participant

In [1]:
from deepeye_analysis_package.getFixations import batch_extract_fixations

# Determine the environment ('Home' or 'Office') and set the data path accordingly
WHERE = 'Home'  # 'Office' or 'Home'

if WHERE == 'Home':
    path_to_data = r'C:/Users/aby600/Dropbox/Appliedwork/CognitiveSolutions/Projects/DeepEye/TechnicalReports/TechnicalReport1/Test_PreferentialViewing/Pilot_PreferentialViewing/withDelay/Approved/data'
else:
    # path = r'D:/Dropbox/Appliedwork/CognitiveSolutions/Projects/DeepEye/TechnicalReports/TechnicalReport1/Test_PreferentialViewing/Pilot_PreferentialViewing/Young/Approved'
    path_to_data =  r'D:/Dropbox/Appliedwork/CognitiveSolutions/Projects/DeepEye/TechnicalReports/TechnicalReport1/Test_PreferentialViewing/Pilot_PreferentialViewing/withDelay/Approved/data'

batch_extract_fixations(path_to_data)

Processing participant 2024_09_26_20_37_08...


c:\Users\aby600\Documents\GitHub\DeepEye_analyze\FixationDetection\I2MC.py:52: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  origData = origData.apply(pd.to_numeric, errors='ignore')
c:\Users\aby600\Documents\GitHub\DeepEye_analyze\FixationDetection\import_funcs.py:124: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  data = data.apply(pd.to_numeric, errors='ignore')





Importing and processing: "C:/Users/aby600/Dropbox/Appliedwork/CognitiveSolutions/Projects/DeepEye/TechnicalReports/TechnicalReport1/Test_PreferentialViewing/Pilot_PreferentialViewing/withDelay/Approved/data\2024_09_26_20_37_08\2024_09_26_20_37_08_record.csv"
	Searching for valid interpolation windows
	Replace interpolation windows with Steffen interpolation
	2-Means clustering started for averaged signal
	Determining fixations based on clustering weight mean for averaged signal and separate eyes + 2*std


I2MC took 3.4128258228302s to finish!


c:\Users\aby600\Documents\GitHub\DeepEye_analyze\deepeye_analyze\lib\site-packages\numpy\lib\_nanfunctions_impl.py:1241: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
c:\Users\aby600\Documents\GitHub\DeepEye_analyze\deepeye_analyze\lib\site-packages\numpy\_core\fromnumeric.py:3904: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
c:\Users\aby600\Documents\GitHub\DeepEye_analyze\deepeye_analyze\lib\site-packages\numpy\_core\_methods.py:147: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


Processing complete.
